<a href="https://colab.research.google.com/github/SakshiMore22/Agastya/blob/main/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
%pip install sentence-transformers -q
from sentence_transformers import SentenceTransformer, util
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.optimizers.legacy import SGD
import random
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
from google.colab import files
uploaded = files.upload()
words=[]
classes = []
documents = []
ignore_words = ['?', '!' ,'.','$','%','(',')','-','.',',','/']
data_file = open('output_file.json').read()
intents = json.loads(data_file)

Saving output_file.json to output_file.json


In [97]:
lst=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        # print(intent)
        documents.append((w, intent['tags']))

        lst.append([w])
        # add to our classes list
        if intent['tags'] not in classes:
            classes.append(intent['tags'])

print(words)
print(lst)
print(documents)
print(classes)

["''", "'d", "'ll", "'m", "'re", "'s", "'ve", '0.5', '0.5mg', '1', '10', '100', '12', '13', '17', '18', '19', '2', '2-3', '20', '29', '3', '3-years', '30', '35', '4', '4-5', '4:30', '4:45', '4th', '50', '5000', '6', '6:00', '7', '8', '99', ':', ';', '``', 'a', 'abandoned', 'abandonment', 'ability', 'able', 'about', 'absolutely', 'abuse', 'abusing', 'abusive', 'academic', 'accepting', 'access', 'accident', 'accomplishment', 'according', 'account', 'achievable', 'achieving', 'acknowledge', 'act', 'acting', 'action', 'actual', 'actually', 'adddiction', 'addiction', 'address', 'adhd', 'adult', 'advice', 'advised', 'affect', 'affected', 'affecting', 'affection', 'afford', 'afraid', 'after', 'afternoon', 'again', 'against', 'age', 'agency', 'ago', 'agoraphobia', 'alcohol', 'alike', 'alive', 'all', 'allen', 'alll', 'almost', 'alone', 'along', 'alot', 'already', 'also', 'although', 'always', 'am', 'amazing', 'an', 'and', 'anger', 'angle', 'angry', 'animal', 'annoyed', 'annoying', 'another', 'a

In [9]:
# lemmaztize and lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

1237 documents
125 classes ['Aagastya-useful', 'Social_anxiety', 'about', 'adddiction_remedy', 'addiction_causes', 'afternoon', 'anxiety', 'anxiety_attacks', 'anxious', 'appearance_anxiety', 'ask', 'casual', 'confidence', 'counseling-fundamentals', 'creation', 'death', 'default', 'depressed', 'depression', 'done', 'empowerment', 'encouragement', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goal_setting', 'goodbye', 'gratitude', 'gratitude_practice', 'greetings', 'happy', 'hate-me', 'hate-you', 'headache_issues', 'help', 'hopefulness', 'howareyou', 'inspiration', 'jokes', 'joy', 'kindness', 'learn-mental-health', 'learn-more', 'location', 'meditation', 'mental-health-fact', 'mental

In [10]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [11]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

<ipython-input-11-dcefa8d881d1>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [12]:
split_index = int(0.8 * len(training))
train_data = training[:split_index]
test_data = training[split_index:]

# Separate features (X) and labels (Y) for training and testing sets
train_x = list(train_data[:, 0])
train_y = list(train_data[:, 1])

test_x = list(test_data[:, 0])
test_y = list(test_data[:, 1])

In [ ]:
print(train_x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.summary()

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               228864    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 125)               8125      
                                                                 
Total params: 245245 (957.99 KB)
Trainable params: 245245 (957.99 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200 , batch_size=10, verbose=1)

# Evaluate the model on the test set
accuracy = model.evaluate(np.array(test_x), np.array(test_y), verbose=1)
print("Test Accuracy:", accuracy[1])

# Save the model
model.save('chatbot_model.h5', hist)

print("Model created and saved")

Epoch 1/200
99/99 [==============================] - 1s 3ms/step - loss: 1.7108 - accuracy: 0.5956
Epoch 2/200
99/99 [==============================] - 0s 4ms/step - loss: 1.6447 - accuracy: 0.6097
Epoch 3/200
99/99 [==============================] - 0s 3ms/step - loss: 1.6431 - accuracy: 0.6087
Epoch 4/200
99/99 [==============================] - 0s 3ms/step - loss: 1.6174 - accuracy: 0.6239
Epoch 5/200
99/99 [==============================] - 0s 4ms/step - loss: 1.5997 - accuracy: 0.6330
Epoch 6/200
99/99 [==============================] - 0s 4ms/step - loss: 1.5865 - accuracy: 0.6269
Epoch 7/200
99/99 [==============================] - 0s 4ms/step - loss: 1.5699 - accuracy: 0.6289
Epoch 8/200
99/99 [==============================] - 0s 4ms/step - loss: 1.5408 - accuracy: 0.6218
Epoch 9/200
99/99 [==============================] - 0s 3ms/step - loss: 1.5440 - accuracy: 0.6229
Epoch 10/200
99/99 [==============================] - 0s 4ms/step - loss: 1.5011 - accuracy: 0.6259
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [84]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import gensim
from gensim.models import Word2Vec
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('output_file.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [45]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [78]:


def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    sentence_words = list(sentence_words)
    print(sentence_words)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            print(model1.wv.similarity(s,w))
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    # print(np.array(bag))
    return(np.array(bag))


In [71]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=True)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.20
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    print(results)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    if ints:
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        print(list_of_intents)
        for i in list_of_intents:
            if(i['tags']== tag):
                result = random.choice(i['responses'])
                break
        return result
    else:
        return "Sorry, I didn't understand that."

def chatbot_response(msg):
    ints = predict_class(msg, model)
    print(ints,"ints")
    res = getResponse(ints, intents)
    print(res)



In [77]:
while True:
  message=input()
  chatbot_response(message)


KeyboardInterrupt: Interrupted by user